In [31]:
import math
import tensorflow as tf
import numpy as np
import pylab as plt

In [32]:
# scale data
def scale(X, X_min, X_max):
    return (X - X_min)/(X_max-X_min)

def process_inputs_from_file(fileName): # Read in data
    inputs = np.loadtxt(fileName, delimiter=' ')
    X, _Y = inputs[:, :NUM_FEATURES], inputs[:, -1].astype(int)
    X = scale(X, np.min(X, axis=0), np.max(X, axis=0))
    _Y[_Y == 7] = 6 # Actually dont have, just in case have error data

    Y = np.zeros((_Y.shape[0], NUM_CLASSES))
    Y[np.arange(_Y.shape[0]), _Y - 1] = 1 #one hot matrix
    return X, Y

def init_weights(feature_no, neuron_no, name, logistic = True):
    # From eg.5.2
    n_in = feature_no
    n_out = neuron_no
    W_values = np.asarray(np.random.uniform(low=-np.sqrt(6. / (n_in + n_out)),
                                            high=np.sqrt(6. / (n_in + n_out)),
                                            size=(n_in, n_out)))
    if logistic == True:
        W_values *= 4
    return(tf.Variable(W_values, dtype=tf.float32, name=name))

def init_bias(neuron_no, name):
    # From eg.5.2
    return(tf.Variable(np.zeros(neuron_no), dtype=tf.float32, name=name))

def setup_cross_entropy(labels, logits):
    return tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=logits)

def setup_correct_prediction(labels, logits):
    return tf.cast(tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1)), tf.float32)

In [33]:
def generate_batch(X_in, y_in, batch_size):
    X = list(X_in)
    y = list(y_in)
    
    if len(X)!=len(y):
        print("Error: len(X)!=len(Y)")
        return None
    batched_X = []
    batched_y = []
    count = 0
    while (len(X) >= batch_size):
        batched_X.append(X[0:batch_size])
        del X[0:batch_size]
        batched_y.append(y[0:batch_size])
        del y[0:batch_size]
        if count % 50 == 0:
            print (count)
        count += 1
    print(count)
    if len(X) != 0:
        remain = batch_size-len(X)
        X.extend(batched_X[0][0:remain])
        y.extend(batched_y[0][0:remain])
        batched_X.append(X)
        batched_y.append(y)
        print("Remain rescaled to", len(X))
    
    return (batched_X, batched_y)

In [34]:
# Initialize Values
NUM_FEATURES = 36
NUM_CLASSES = 6
NUM_HIDDEN = 10

LEARNING_RATE = 0.01
EPOCHS = 5000
BATCH_SIZE = 32
NUM_NEURONS = 10
SEED = 10
BETA = pow(10, -6)
np.random.seed(SEED)

TRAIN_FILE_NAME = 'sat_train.txt'
TEST_FILE_NAME = 'sat_test.txt'

In [35]:
trainX, trainY = process_inputs_from_file(TRAIN_FILE_NAME)
testX, testY = process_inputs_from_file(TEST_FILE_NAME)

x = tf.placeholder(tf.float32, [None, NUM_FEATURES], name='x')
d = tf.placeholder(tf.float32, [None, NUM_CLASSES], name='d')

with tf.variable_scope("Hidden_layer"):
    W = init_weights(NUM_FEATURES, NUM_HIDDEN, name="Weight_1")
    b = init_bias(NUM_HIDDEN, name="Bias_1")
    z = tf.matmul(x, W) + b #syn_input_1
    h = tf.nn.sigmoid(z) #out_1

with tf.variable_scope("Output_layer"):
    V = init_weights(NUM_HIDDEN, NUM_CLASSES, name="Weight_2")
    c = init_bias(NUM_CLASSES, name="Bias_2" )
    u = tf.matmul(h, V) + c #syn_out_2
    y = u #out_2  # Consider to change to sigmoid

cross_entropy = setup_cross_entropy(labels=d, logits=y)
regularization = tf.nn.l2_loss(V) + tf.nn.l2_loss(W) 
J = tf.reduce_mean(cross_entropy + BETA * regularization)

optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE)
train_op = optimizer.minimize(J)

correct_prediction = setup_correct_prediction(labels=d, logits=y)
accuracy = tf.reduce_mean(correct_prediction)

## Train & Test

In [36]:
batch_X, batch_Y = generate_batch(trainX, trainY, BATCH_SIZE)

0
50
100
138
Remain rescaled to 32


In [37]:
saver = tf.train.Saver()
model_name = "model-PartA-5000-32"
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    train_acc = []

    print("********* BEFORE **********")
    print(W.eval())
    print(b.eval())
    print(V.eval())
    print(c.eval())
    print("***************************")
    
    for i in range(EPOCHS):    
        
        # ============= RANDOM BATCH ============= #
    #     for j in range(BATCH_SIZE):
    #         np.random.seed()
    #         rand_index = np.random.choice(a=len(trainX), size=BATCH_SIZE)
    #         x_batch = trainX[rand_index]  # Transpose to the correct shape
    #         y_batch = trainY[rand_index]
    #         train_op.run(feed_dict={x: x_batch, d: y_batch})

    #     train_acc.append(accuracy.eval(feed_dict={x: x_batch, d: y_batch}))
        # ======================================== #

        
        
        # ============= SEQUENCED BATCH ============= #
        for j in range(len(batch_X)): # batch_X[j]: One batch with 32 records
            train_op.run(feed_dict={x: batch_X[j], d: batch_Y[j]})
        train_acc.append(accuracy.eval(feed_dict={x: batch_X[j], d: batch_Y[j]}))
        # ======================================== #
        

        if i % 100 == 0:
            print('iter %d: accuracy %g'%(i, train_acc[i]))

    print("********* AFTER **********")
    print(W.eval())
    print(b.eval())
    print(V.eval())
    print(c.eval())
    print("**************************")

    save_path = saver.save(sess, "/save/"+model_name+".ckpt")
    print("Model saved in path: %s" % save_path)

********* BEFORE **********
[[ 0.783916   -1.3846724   0.38614455  0.7188592  -0.00431363 -0.7951342
  -0.872375    0.7527411  -0.956025   -1.1893935 ]
 [ 0.5355528   1.3099715  -1.4332227   0.03522662  0.90324336  0.3251171
   0.64070886 -0.60132426  1.2070583   0.6199653 ]
 [ 0.12292176 -1.0338639  -0.36595154  0.5031174  -0.16805911 -0.19065076
   0.34025946  0.03795981  0.43453664  0.29192784]
 [ 0.8818693   0.06254426  1.180693   -0.522274   -1.1832696  -0.57582945
  -1.1152997   0.94964594 -1.3091346   0.36487648]
 [ 0.1374888   0.9225033  -0.869819    1.0310335  -0.42861417  0.7357435
  -0.5895198   1.1092925  -0.50414234 -0.96785635]
 [-0.310511   -1.1745988   0.92775786 -1.0079126  -0.33482355  1.283585
   1.408877   -0.12624754  0.9422538  -0.71834487]
 [ 0.28133205  1.1638858   0.09984691  0.26061523 -1.331135   -0.4126391
  -1.2146072  -0.562077   -0.48909602  0.7911671 ]
 [-1.3291776  -0.20371546 -0.5347245   0.39435846 -0.4439431  -1.3201108
   1.0976739   0.76057065  1.0

In [38]:
# print("********* RESULT **********")
# print(W.eval())
# print(b.eval())
# print(V.eval())
# print(c.eval())
# print("***************************")
with tf.Session() as sess:
#     tf.global_variables_initializer().run()
    output_2_, accuracy_ = sess.run([y, accuracy], feed_dict={x: testX, d: testY})
    print(output_2_, '\n',accuracy_)

FailedPreconditionError: Attempting to use uninitialized value Hidden_layer_4/Weight_1
	 [[Node: Hidden_layer_4/Weight_1/read = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Hidden_layer_4/Weight_1)]]
	 [[Node: Mean_9/_5 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_33_Mean_9", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Hidden_layer_4/Weight_1/read', defined at:
  File "C:\Users\lzcai\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\lzcai\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\lzcai\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\lzcai\Anaconda3\lib\site-packages\traitlets\config

In [23]:
testX.shape

(2000, 36)

## Alt: Read In Model & Test

In [30]:
model_name = "model-PartA-5000-32-2"
saver = tf.train.Saver()
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "/save/"+model_name+".ckpt")
    print("Model restored.")
    # Check the values of the variables
    print("W : %s" % W.eval())
    print("b : %s" % b.eval())
    
#     tf.global_variables_initializer().run()
    output_2_, accuracy_ = sess.run([f, accuracy], feed_dict={x: testX, d: testY})
    print(output_2_, '\n',accuracy_)

INFO:tensorflow:Restoring parameters from /save/model-PartA-5000-32-2.ckpt


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key Hidden_layer_3/Bias_1 not found in checkpoint
	 [[Node: save_6/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_6/Const_0_0, save_6/RestoreV2/tensor_names, save_6/RestoreV2/shape_and_slices)]]

Caused by op 'save_6/RestoreV2', defined at:
  File "C:\Users\lzcai\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\lzcai\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\lzcai\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\lzcai\A

## Plot Graph

In [ ]:
# plot learning curves
plt.figure(1)
plt.plot(range(EPOCHS), train_acc)
plt.xlabel(str(EPOCHS) + ' iterations')
plt.ylabel('Train accuracy')
plt.savefig("PartA-5000-32-random.png")
plt.show()